In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
# from config_profile import args
# from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from scipy.optimize import linear_sum_assignment
import sys
import matplotlib.pyplot as plt

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

q = torch.load("./keypoint_descriptor/query_description.pt")  # torch.Size([35, 30, 128])
d = torch.load("./keypoint_descriptor/data_description.pt")   # torch.Size([140, 30, 128])
p_point = torch.load("keypoints_query.pt")
q_point = torch.load("keypoints_images.pt")
F = torch.load("./fundamental_matrix.pt")

print(q.shape)
print(d.shape)
print("p_keypoints: ", p_point.shape)
print("q_keypoints: ", q_point.shape)
print(F.shape)

torch.Size([35, 30, 128])
torch.Size([140, 30, 128])
p_keypoints:  torch.Size([35, 30, 2])
q_keypoints:  torch.Size([140, 30, 2])
torch.Size([35, 140, 3, 3])


In [9]:
data = np.zeros((30, 30))
similarity_list_one = np.zeros((35, 140))
cost_matrix = torch.tensor(data)
pair_matrix = data.tolist()

# for i in range(5):
#     for j in range(140):
for i in range(len(q)):                 # 35
#     for j in range(len(d)):             # 140  
    for j in range(1):
        s_one = 0
        s_many = 0
        
        for k in range(len(q[0])):      # 30
            for l in range(len(d[0])):  # 30
                pair_matrix[i][j] = (p_point[i][k].tolist(), q_point[j][l].tolist())
                cost_matrix[k][l] = torch.dist(q[i][k], d[j][l], p=2)
                
            print("\r{},{},{}".format(i+1, j+1, k+1), end='')
            sys.stdout.flush()
        
        # one2one
        cost_matrix_np = cost_matrix.detach().numpy()
        row_ind, col_ind = linear_sum_assignment(cost_matrix_np)
        
        # C: one-to-one matching pairs
        c = []
        for i in range(len(col_ind)):
            c.append(pair_matrix[i][col_ind[i]])
            c[i][0].append(1)
            c[i][1].append(1)
        # print("\nC:\n", c)    
        print("shape of C:", np.shape(c))    #
             
        
        for m in range(len(cost_matrix_np[0])):       # 30
            s_one += np.exp(-cost_matrix_np[m][col_ind[m]])
        similarity_list_one[i][j] = s_one
        

        



1,1,30

TypeError: 'float' object is not subscriptable